**30E03000 - Data Science for Business I (2021)**

# Tutorial 4: Feature Selection for Sparsity

### Acknowledgement

This tutorial on Subset Selection, Ridge Regression, and Lasso is based on pp. 244-247 and pp. 251-255 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. The Python code has been adapted from R. Jordan Crouser at Smith College.

### Overview 

In this tutorial, you will get acquainted with the **bias-variance trade-off** problem in linear regression and how it can be solved with (sequential) feature selection algorithms and regularization.

<img src="Images/variancebias.png" alt="Drawing" style="width: 700px;"/>

The motivation behind feature selection algorithms is to automatically select a subset of features that is most relevant to the problem. The goal of feature selection is two-fold: We want to **improve the computational efficiency** and **reduce the generalization error** of the model by removing irrelevant features or noise.

In this tutorial, we will discuss:
- Best Subset Selection
- Sequential Forward Selection (SFS)
- Sequential Backward Selection (SBS)
- Ridge Regression
- Lasso

### Problem setting

In this tutorial, we are working with Baseball data that was collected over 2 seasons. We wish to **predict a baseball player’s salary** based on various statistics describing the performance in the previous year:

- AtBat: Number of times at bat (season 1)
- Hits: Number of hits (season 1)
- HmRun: Number of home runs (season 1)
- Runs: Number of runs (season 1)
- RBI: Number of runs batted (season 1)
- Walks: Number of walks (season 1)
- Years: Number of years in the major leagues
- CAtBat: Number of times at bat during his career
- CHits: Number of hits during his career
- CHmRun: Number of home runs during his career
- CRuns: Number of runs during his career
- CRBI: Number of runs batted in during his career
- CWalks: Number of walks during his career
- League: A factor with levels A and N indicating player's league (season 1)
- Division: A factor with levels E and W indicating player's division (season 1)
- PutOuts: Number of put outs (season 1)
- Assists: Number of assists (season 1)
- Errors: Number of errors (season 1)
- Salary: 2nd season annual salary on opening day in thousands of dollars
- NewLeague: A factor with levels A and N indicating player's league at the beginning of season 2

**Source:** StatLib library at Carnegie Mellon University. 

<table><tr>
<td> <img src="Images/hit.jpeg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="Images/court.jpg" alt="Drawing" style="width: 450px;"/> </td>
    <td> <img src="Images/ball.jpeg" alt="Drawing" style="width: 270px;"/> </td>
</tr></table>

## Import libraries

In [15]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [16]:
%config Completer.use_jedi = False

## Load Data

In [17]:
data = pd.read_csv('hitters.csv')
data.head().style #Call the head() function to see how the dataset looks (.style() shows all columns)

,player,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,nan,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [18]:
# Print the dimensions of the original Hitters data (322 rows x 20 columns)
print("Dimensions of original data:", data.shape)

Dimensions of original data: (322, 21)


Calling `data.info()`, we note that the `Salary` **variable is missing** for 59 players (263/322 entries non-null). Before doing any kind of feature selection, we have to remove the NaN values.

## Preprocessing

As usual, we preprocess the data to allow for the subsequent modeling steps:
- remove all entries where (at least) one variable is NaN, using Panda's `dropna()` function
- perfrom one-hot-encoding for all string variables, using Panda's `get_dummies()` function
- drop columns with no information value (player names) and columns that have been one-hot-encoded using `drop()`
- merge the data with the one-hot-encoded dummy values, using `concat()`

In [19]:
#Drop rows where (at least) one variable is NaN -> removes Salary = NaN values
data = data.dropna() 
data.shape

(263, 21)

In [20]:
# Drop the player column (no predictive value)
data = data.drop(['player'], axis=1)

<font color='red'>**Beware of multicollinearity when dealing with Regression models**</font>

In [21]:
#Use panda`s "get_dummies()" function to one-hot-encode the string variables into a new dataframe 
pd.get_dummies(data[['League', 'Division', 'NewLeague']]).head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [22]:
#Use the drop_first=True argument to fix the multicollinearity issue when performing one-hot encoding
data = pd.get_dummies(data, columns=['League', 'Division', 'NewLeague'], prefix=['League', 'Division', 'NewLeague'],
                      drop_first=True)

In [23]:
data.head().style

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475,1,1,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480,0,1,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500,1,0,1
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.5,1,0,1
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,750,0,1,0


In [24]:
# Print the dimensions of the modified Hitters data (263 rows x 20 columns)
print("Dimensions of modified data:", data.shape)

Dimensions of modified data: (263, 20)


Next, we create a feature dataframe `X` and a target vector `y`.

In [25]:
y = data['Salary'].values #define target variable 
X = data.loc[:, data.columns != 'Salary'] #define feature matrix

In [12]:
X = sm.add_constant(X)

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


## Data split

We split the data 70:30 into training and testing:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Why do you need feature selection/sparse models?

In [ ]:
#exog = sm.add_constant(X_train)
result = sm.OLS(y_train, X_train).fit()
print(result.summary())

In [ ]:
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, result.predict(X_train)), 0)))
print('Test MSE:    ' + str(round(mean_squared_error(y_test, result.predict(X_test)), 0)))

<font color='red'>**Problem: big performance difference between train and test!**</font>

--> we want to find a sparse model (fewer features) in the hopes that such a model will generalize better to unseen test data.

<img src="Images/variancebias_edit.png" alt="Drawing" style="width: 700px;"/>



## Option 1: Best Subset Selection (the Brute Force approach)

First, we will perfrom Best Subset Selection by identifying the best model that contains a given number of features, where **best** is quantified using the [Residual Sum of Squared (RSS)](https://en.wikipedia.org/wiki/Residual_sum_of_squares). We'll define a helper function `processSubset()` to outputs the best set of variables for each model size:

In [ ]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    
    features = list(feature_set)
    if 'const' not in features:
        features.append('const')
    
    exog = X_train[features]
    exog_test = X_test[features]

    model = sm.OLS(y_train, exog)
    regr = model.fit()
    
    train_MSE = round(mean_squared_error(y_train, regr.predict(exog)), 0)
    test_MSE = round(mean_squared_error(y_test, regr.predict(exog_test)), 0)
    
    RSS = ((regr.predict(exog) - y_train) ** 2).sum()
    return {"model":regr, "RSS":RSS, "Train MSE": train_MSE, "Test MSE": test_MSE}

In [ ]:
def getBest(k):
    
    tic = time.time()
    
    results = []
        
    for combo in itertools.combinations(X.columns[1:-1], k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].idxmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (round(toc-tic,3)), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

 Now we want to call that function for each number of features $k$:

In [ ]:
# Could take quite awhile to complete...

models_best = pd.DataFrame(columns=["model", "RSS", "Train MSE", "Test MSE"])

tic = time.time()
for i in range(1,7): #between 1 and 7 features
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Efficiency matters: the Best Subset selection algorithm is computationally heavy. The CS Jupyter Hub is not able to handle more than 6 predictors! 

Now we have one big Dataframe `models_best` that contains the best models for each $k$ value along with their RSS, Train MSE, and Test MSE:

In [ ]:
models_best

We can get the details of each model by calling the `summary()` function:

In [ ]:
print(models_best.loc[2, "model"].summary())

This output indicates that the best **two-variable model** contains `Hits` and `CRBI`. 

Printing the above summery for all 6 models would quickly clutter the notebook. Instead of calling the `summary()` function, we can direcly access the models' attributes we are interested in, e.g. adjusted $R^2$, AIC, and BIC. We can examine these to try to select the best overall model. Let's start by looking at $R^2$ across all our models:

In [ ]:
#models_best.loc[2, "model"].rsquared

# Gets the second element from each row ('model') and pulls out its rsquared attribute
models_best.apply(lambda row: row[0].rsquared, axis=1)

As expected, the $R^2$ statistic increases monotonically as more variables are included.

Plotting MSE, adjusted $R^2$, AIC, and BIC for all of the models at once will help us decide which model to select:

In [ ]:
def plotFeatureSelection(model):
    plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

    # Set up a 2x2 grid so we can look at 4 plots at once
    plt.subplot(2, 2, 1)

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The idxmax() function can be used to identify the location of the maximum point of a vector
    #plt.plot(model["RSS"])
    #plt.xticks(model.index)
    #plt.xlabel('# Predictors')
    #plt.ylabel('RSS')

    plt.plot(model["Train MSE"], label='train')
    plt.plot(model["Test MSE"], label='test')
    plt.xticks(model.index)
    plt.xlabel('# Predictors')
    plt.ylabel('MSE')
    plt.legend()
    
    
    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The idxmax() function can be used to identify the location of the maximum point of a vector

    rsquared_adj = model.apply(lambda row: row[0].rsquared_adj, axis=1)

    plt.subplot(2, 2, 2)
    plt.plot(rsquared_adj)
    plt.plot(rsquared_adj.idxmax(), rsquared_adj.max(), "or")
    plt.xticks(model.index)
    plt.xlabel('# Predictors')
    plt.ylabel('adjusted rsquared')

    # We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
    aic = model.apply(lambda row: row[0].aic, axis=1)

    plt.subplot(2, 2, 3)
    plt.plot(aic)
    plt.plot(aic.idxmin(), aic.min(), "or")
    plt.xticks(model.index)
    plt.xlabel('# Predictors')
    plt.ylabel('AIC')

    bic = model.apply(lambda row: row[0].bic, axis=1)

    plt.subplot(2, 2, 4)
    plt.plot(bic)
    plt.plot(bic.idxmin(), bic.min(), "or")
    plt.xticks(model.index)
    plt.xlabel('# Predictors')
    plt.ylabel('BIC')

    plt.tight_layout();

In [ ]:
plotFeatureSelection(models_best)

We see that according to adjusted $R^2$ something, AIC, and BIC the 6-variable model is best. Again, no one measure is going to give us an entirely accurate picture... but they all agree that a model with 5 or fewer predictors is insufficient.

In summary, the Best Subset Selection approach resembles a brute force approach in the way that it simply tries all possible combinations. It becomes computationally heavy very quickly, even on small datasets (which is why we did not explore beyond k = 7 in the above example). 

## Option 2: Forward Selection

A more elegant approach is Forward Selection (FWS). With a slight modification, The Best Subset Selection code can be used for Forward Selection:

In [ ]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns[1:-1] if p not in predictors]
    #print(remaining_predictors)
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
        #print(predictors+[p])
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].idxmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (round(toc-tic,3)), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
models_fwd = pd.DataFrame(columns=["model", "RSS", "Train MSE", "Test MSE"])



tic = time.time()
predictors = ['const']
#predictors.append(X.)

for i in range(1,len(X.columns[1:-1])+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Forward-stepwise selection is a greedy algorithm, producing a nested sequence of models.

The forward selection approach runs sooooo much faster, even when evaluating models with large k values!

<img src="Images/vegeta.jpg" alt="Drawing" style="width: 350px;"/>

As in the Best Subset Selection, Forward Selection suggests that the best 2-variable model includes `Hits` and `CRBI` (not shown here). Additionally, we compare the best 6-variable model for each approach:

In [ ]:
print(models_best.loc[6, "model"].summary())
print()
print(models_fwd.loc[6, "model"].summary())

Both methods suggest different variables. Moreover, Forward Selection found the subset **significantly** faster!

Plotting MSE, adjusted $R^2$, AIC, and BIC for all of the models at once will help us decide which model to select:

In [ ]:
plotFeatureSelection(models_fwd)

Beware: once a variable is including in the model it will stay there!

## Option 3: Backwards Selection

We recycle the Forward Selection code, but loop through the predictors in reverse to perform Backwards Selection:

In [ ]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-2):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].idxmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (round(toc-tic,3)), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
models_bwd = pd.DataFrame(columns=["model", "RSS", "Train MSE", "Test MSE"])#, 
                          #index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns[1:-1]

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

In [ ]:
print(models_bwd.loc[6, "model"].summary())

We plot the best 6-variable models for Best Subset, Forwards, and Backwards Selection:

In [ ]:
for model, name in zip([models_best, models_fwd, models_bwd], ['Best Subset', 'Forward Selection', 'Backward Selection']):
    print("\n------------")
    print(name)
    print("------------")
    print(model.loc[6, "model"].params.sort_index())

Plotting MSE, adjusted $R^2$, AIC, and BIC for all of the models at once will help us decide which model to select:

In [ ]:
plotFeatureSelection(models_bwd)

<img src="Images/variancebias_edit_3.png" alt="Drawing" style="width: 700px;"/>

## Option 4: Ridge Regression

Ridge estimates are found by minimizing (from slides):

<img src="Images/ridge.png" alt="Drawing" style="width: 500px;"/>

- As with least squares, ridge regression seeks coefficient estimates that fit the data well, by making the RSS small.
- **The second term is a penalty that shrinks the coefficients towards zero**
- Though not immediately obvious, shrinking can help to reduce variance
- The estimates are very sensitive to the “tuning parameter” lambda -> needs to be found separately using cross validation!
- The tuning parameter controls the relative impact of RSS and penalty on the regression coefficient estimates.

We will use a `sklearn` package in order to perform ridge regression. Specifically, we will be using `Ridge()` to fit ridge regression models and its cross-validated counterpart `RidgeCV()`.


The `Ridge()` function has an alpha argument **($\lambda$, but with a different name!)** that is used to tune the model. We'll generate an array of alpha values ($\lambda$!!!) ranging from very big to very small, essentially covering the full range of scenarios from the null model containing only the intercept, to the least squares fit:

In [ ]:
np.set_printoptions(suppress=True, precision=2)

In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5
alphas

In [ ]:
plt.plot(alphas)
plt.ylabel('alpha value');

Associated with each alpha value is a vector of ridge regression coefficients, which we'll store in a matrix `coefs`. In this case, it is a $19 \times 100$ matrix, with 19 rows (one for each predictor) and 100 columns (one for each value of alpha). Remember that we'll want to standardize the variables so that they are on the same scale. To do this, we can use the `normalize = True` parameter:

In [ ]:
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)
    
np.shape(coefs)

We expect the coefficient estimates to be much smaller, in terms of $l_2$ norm, when a large value of alpha is used, as compared to when a small value of alpha is used. Let's plot and find out:

In [ ]:
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights');

Next we fit a ridge regression model on the training set, and evaluate its MSE on the test set, using $\lambda = 4$ (the lambda value is **chosen randomly** for illustrative purposes. We'll cover how to pick an optimal value in a second):

In [ ]:
ridge2 = Ridge(alpha = 4, normalize = True).fit(X_train, y_train) # Fit a ridge regression on the training data
pred2 = ridge2.predict(X_test) # Use this model to predict the test data
print(pd.Series(ridge2.coef_, index = X.columns)) # Print coefficients

print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, ridge2.predict(X_train)), 0)))
print('\nTest MSE:     ' + str(round(mean_squared_error(y_test, ridge2.predict(X_test)), 0)))

<img src="Images/variancebias_edit_4.png" alt="Drawing" style="width: 700px;"/>

The test MSE when alpha = 4 is 129,245. Now let's see what happens if we use a huge value of alpha, say $10^{10}$:

In [ ]:
ridge3 = Ridge(alpha = 10**10, normalize = True)
ridge3.fit(X_train, y_train)             # Fit a ridge regression on the training data
pred3 = ridge3.predict(X_test)           # Use this model to predict the test data
print(pd.Series(ridge3.coef_, index = X.columns)) # Print coefficients
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, ridge3.predict(X_train)), 0)))
print('\nTest MSE:     ' + str(round(mean_squared_error(y_test, ridge3.predict(X_test)), 0)))

This big penalty shrinks the coefficients to a very large degree, essentially reducing to a model containing just the intercept. This **over-shrinking makes the model more biased**, resulting in a higher MSE.

<img src="Images/variancebias_edit_5.png" alt="Drawing" style="width: 700px;"/>

Okay, so fitting a ridge regression model with alpha = 4 leads to a much lower test MSE than fitting a model with just an intercept. We now check whether there is any benefit to performing ridge regression with alpha = 4 instead of just performing least squares regression. Recall that least squares is simply ridge regression with alpha = 0.

In [ ]:
ridge4 = Ridge(alpha = 0, normalize = True).fit(X_train, y_train)             # Fit a ridge regression on the training data
pred = ridge4.predict(X_test)            # Use this model to predict the test data
print(pd.Series(ridge4.coef_, index = X.columns)) # Print coefficients
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, ridge4.predict(X_train)), 0)))
print('\nTest MSE:     ' + str(round(mean_squared_error(y_test, ridge4.predict(X_test)), 0)))

It looks like we are indeed improving over regular least-squares!

Instead of arbitrarily choosing alpha $ = 4$, it would be better to use cross-validation to choose the tuning parameter alpha. We can do this using the cross-validated ridge regression function, `RidgeCV()`. By default, the function performs generalized cross-validation (an efficient form of LOOCV), though this can be changed using the argument `cv`.

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_

Therefore, we see that the value of alpha that results in the smallest cross-validation
error is 0.57. What is the test MSE associated with this value of
alpha?

In [ ]:
ridge5 = Ridge(alpha = ridgecv.alpha_, normalize = True).fit(X_train, y_train)
mean_squared_error(y_test, ridge5.predict(X_test))
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, ridge5.predict(X_train)), 0)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, ridge5.predict(X_test)), 0)))

Unlike subset selection, which will generally select models that involve just a subset of the variables, 
<font color='red'>**ridge regression will include all p predictors in the final model!** </font>

## Option 5: LASSO

Similar to ridge regression, but the key difference in behavior follows from penalty:

<img src="Images/lasso.png" alt="Drawing" style="width: 500px;"/>

We saw that ridge regression with a wise choice of alpha can outperform least
squares as well as the null model on the Hitters data set. We now ask
whether the lasso can yield either a more accurate or a more interpretable
model than ridge regression. In order to fit a lasso model, we'll
use the `Lasso()` function; however, this time we'll need to include the argument `max_iter = 10000`.
Other than that change, we proceed just as we did in fitting a ridge model:

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(X_train.astype(float)), y_train)
    coefs.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights');

Notice that in the coefficient plot that depending on the choice of tuning
parameter, some of the coefficients are exactly equal to zero. We now
perform 10-fold cross-validation to choose the best alpha, refit the model, and compute the associated test error:

In [ ]:
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(X_train, y_train)
print('\nTraining MSE: ' + str(round(mean_squared_error(y_train, lasso.predict(X_train)), 0)))
print('\nTest MSE:    ' + str(round(mean_squared_error(y_test, lasso.predict(X_test)), 0)))

This is substantially lower than the test set MSE of the null model and of
least squares, and only a little worse than the test MSE of ridge regression with alpha
chosen by cross-validation.

**However, the lasso has a substantial advantage over ridge regression in that the resulting coefficient estimates are sparse.** Here we see that 13 of
the 19 coefficient estimates are exactly zero:

In [ ]:
# Some of the coefficients are now reduced to exactly zero.
pd.Series(lasso.coef_, index=X.columns)

## Ridge vs. Lasso

<img src="Images/ridgevslasso.png" alt="Drawing" style="width: 800px;"/>

## Final throughts

- dont use the forward and backwards selection code that was used in this tutorial! It is simple but inefficient!
- Forward selection using mlxtend -> Sequential Feature Selector: https://www.kdnuggets.com/2018/06/step-forward-feature-selection-python.html